In [36]:
import pandas as pd
import requests
import json
from config import g_key
import gmaps
gmaps.configure(api_key=g_key)

In [6]:
cali_city = pd.read_csv("Clean Resources\home_value_california_city_unpivoted_dropna.csv")
cali_city

,Unnamed: 0,RegionID,SizeRank,RegionName,RegionType,StateName,State,Metro,CountyName,date,price
0,1,12447,1,Los Angeles,City,CA,CA,Los Angeles-Long Beach-Anaheim,Los Angeles County,1/31/2000,237081
1,8,54296,8,San Diego,City,CA,CA,San Diego-Carlsbad,San Diego County,1/31/2000,231226
2,11,33839,11,San Jose,City,CA,CA,San Jose-Sunnyvale-Santa Clara,Santa Clara County,1/31/2000,390074
3,14,20330,14,San Francisco,City,CA,CA,San Francisco-Oakland-Hayward,San Francisco County,1/31/2000,463867
4,30,20288,30,Sacramento,City,CA,CA,Sacramento--Roseville--Arden-Arcade,Sacramento County,1/31/2000,134964
...,...,...,...,...,...,...,...,...,...,...,...
79389,2354119,17442,2573,Chowchilla,City,CA,CA,Madera,Madera County,10/31/2008,147831
79390,2354191,7662,2651,Valley Center,City,CA,CA,San Diego-Carlsbad,San Diego County,10/31/2008,512728
79391,2354196,16760,2656,American Canyon,City,CA,CA,Napa,Napa County,10/31/2008,396327
79392,2354214,6485,2676,Phelan,City,CA,CA,Riverside-San Bernardino-Ontario,San Bernardino County,10/31/2008,218858


In [27]:
group_cali_city = cali_city.groupby(["RegionName"],sort=False, as_index=False).mean()
group_cali_city = group_cali_city[["RegionName", "RegionID", "price"]]
group_cali_city

,RegionName,RegionID,price
0,Los Angeles,12447.0,452810.952830
1,San Diego,54296.0,432342.409524
2,San Jose,33839.0,615362.358491
3,San Francisco,20330.0,743077.783019
4,Sacramento,20288.0,262840.500000
...,...,...,...
798,Marina del Rey,53162.0,558521.250000
799,Laytonville,5577.0,320302.125000
800,Pala,53784.0,705532.125000
801,Manchester,35859.0,496985.125000


In [28]:
#api call
cali_coord_df = group_cali_city.apply(
    lambda row: requests.get(f"https://maps.googleapis.com/maps/api/geocode/json?address={row.RegionName}%20CA&key={g_key}").json()["results"][0]["geometry"]["location"], axis =1, result_type="expand")
cali_coord_df

,lat,lng
0,34.052234,-118.243685
1,32.715738,-117.161084
2,37.338208,-121.886329
3,37.774929,-122.419415
4,38.581572,-121.494400
...,...,...
798,33.980289,-118.451745
799,39.688212,-123.482794
800,33.365217,-117.076474
801,38.969916,-123.686929


In [34]:
cali_df_with_coords = group_cali_city.join(cali_coord_df)
cali_df_with_coords

,RegionName,RegionID,price,lat,lng
0,Los Angeles,12447.0,452810.952830,34.052234,-118.243685
1,San Diego,54296.0,432342.409524,32.715738,-117.161084
2,San Jose,33839.0,615362.358491,37.338208,-121.886329
3,San Francisco,20330.0,743077.783019,37.774929,-122.419415
4,Sacramento,20288.0,262840.500000,38.581572,-121.494400
...,...,...,...,...,...
798,Marina del Rey,53162.0,558521.250000,33.980289,-118.451745
799,Laytonville,5577.0,320302.125000,39.688212,-123.482794
800,Pala,53784.0,705532.125000,33.365217,-117.076474
801,Manchester,35859.0,496985.125000,38.969916,-123.686929


In [42]:
# Plot Heatmap
LatLong = cali_df_with_coords[["lat","lng"]]
fig = gmaps.figure(layout={'width': '400px', 'height': '500px'})
weight = cali_df_with_coords["price"]
heatmap_layer = gmaps.heatmap_layer(LatLong, weights=weight)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='500px', width='400px'))